# Lab 12-3 many to many
### simple pos-tagger training 
* many to many
* variable input sequence length

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
%matplotlib inline

print(tf.__version__)

1.12.0


### Prepairing dataset

In [2]:
sentences = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]
pos = [['pronoun', 'verb', 'adjective'],
     ['noun', 'verb', 'adverb', 'adjective'],
     ['noun', 'verb', 'determiner', 'noun', 'preposition', 'adjective', 'noun'],
     ['noun', 'verb', 'adverb', 'adjective', 'verb']]

### Preprocessing dataset

In [3]:
# word의 dictionary
word_list = sum(sentences, [])
word_list = sorted(set(word_list))
word_list = ['<pad>'] + word_list
word2idx = {word : idx for idx, word in enumerate(word_list)}
idx2word = {idx : word for idx, word in enumerate(word_list)}

print(word2idx)
print(idx2word)
print(len(idx2word))

{'<pad>': 0, 'I': 1, 'a': 2, 'changing': 3, 'deep': 4, 'difficult': 5, 'fast': 6, 'feel': 7, 'for': 8, 'framework': 9, 'hungry': 10, 'is': 11, 'learning': 12, 'tensorflow': 13, 'very': 14}
{0: '<pad>', 1: 'I', 2: 'a', 3: 'changing', 4: 'deep', 5: 'difficult', 6: 'fast', 7: 'feel', 8: 'for', 9: 'framework', 10: 'hungry', 11: 'is', 12: 'learning', 13: 'tensorflow', 14: 'very'}
15


In [4]:
# pos (part of speech)의 dictionary
pos_list = sum(pos, [])
pos_list = sorted(set(pos_list))
pos_list = ['<pad>'] + pos_list
pos2idx = {pos : idx for idx, pos in enumerate(pos_list)}
idx2pos = {idx : pos for idx, pos in enumerate(pos_list)}

print(pos2idx)
print(idx2pos)
print(len(pos2idx))

{'<pad>': 0, 'adjective': 1, 'adverb': 2, 'determiner': 3, 'noun': 4, 'preposition': 5, 'pronoun': 6, 'verb': 7}
{0: '<pad>', 1: 'adjective', 2: 'adverb', 3: 'determiner', 4: 'noun', 5: 'preposition', 6: 'pronoun', 7: 'verb'}
8


In [5]:
max_sequence = 10
x_data = list(map(lambda sentence : [word2idx.get(token) for token in sentence], sentences))
x_data = pad_sequences(sequences = x_data, maxlen = max_sequence,padding='post')
x_data_mask = ((x_data != 0) * 1).astype(np.float32)
x_data_len = np.array(list(map(lambda sentence : len(sentence), sentences)))

y_data = list(map(lambda sentence : [pos2idx.get(token) for token in sentence], pos))
y_data = pad_sequences(sequences = y_data, maxlen = max_sequence,padding='post')
y_data = np.expand_dims(y_data, -1)

In [6]:
# input
tr_dataset = tf.data.Dataset.from_tensor_slices((x_data,y_data,x_data_mask))
tr_dataset = tr_dataset.batch(batch_size = 2).repeat()

print(tr_dataset)

<RepeatDataset shapes: ((?, 10), (?, 10, 1), (?, 10)), types: (tf.int32, tf.int32, tf.float32)>


### Creating model

In [7]:
num_classes = len(pos2idx)
hidden_dim = 10

input_dim = len(word2idx)
output_dim = len(word2idx)
one_hot = np.eye(len(word2idx))

model = Sequential()
model.add(layers.Embedding(input_dim=input_dim, output_dim=output_dim, mask_zero=True,
                                 trainable=False, input_length=max_sequence,
                                 embeddings_initializer=keras.initializers.Constant(one_hot)))
model.add(layers.SimpleRNN(units=hidden_dim, return_sequences=True))
model.add(layers.TimeDistributed(keras.layers.Dense(units=num_classes, activation='softmax')))

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 15)            225       
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 10, 10)            260       
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 8)             88        
Total params: 573
Trainable params: 348
Non-trainable params: 225
_________________________________________________________________


### Training model

In [9]:
# Creating a training operation
lr = .1
opt = tf.train.AdamOptimizer(learning_rate = lr)
model.compile(optimizer=opt,
              loss=keras.losses.sparse_categorical_crossentropy,
              sample_weight_mode="temporal")

In [10]:
model.fit(tr_dataset, epochs=30, steps_per_epoch=4//2, verbose=1)

Epoch 1/30
2/2 [==============================] - 0s 118ms/step - loss: 1.8037
Epoch 2/30
2/2 [==============================] - 0s 5ms/step - loss: 1.0356
Epoch 3/30
2/2 [==============================] - 0s 7ms/step - loss: 0.5860
Epoch 4/30
2/2 [==============================] - 0s 6ms/step - loss: 0.3096
Epoch 5/30
2/2 [==============================] - 0s 5ms/step - loss: 0.1684
Epoch 6/30
2/2 [==============================] - 0s 6ms/step - loss: 0.0922
Epoch 7/30
2/2 [==============================] - 0s 5ms/step - loss: 0.0480
Epoch 8/30
2/2 [==============================] - 0s 6ms/step - loss: 0.0275
Epoch 9/30
2/2 [==============================] - 0s 4ms/step - loss: 0.0150
Epoch 10/30
2/2 [==============================] - 0s 7ms/step - loss: 0.0089
Epoch 11/30
2/2 [==============================] - 0s 3ms/step - loss: 0.0060
Epoch 12/30
2/2 [==============================] - 0s 4ms/step - loss: 0.0044
Epoch 13/30
2/2 [==============================] - 0s 3ms/step - loss: 

### Checking performance

In [11]:
yhat = model.predict(x_data)
yhat = np.argmax(yhat, axis=-1)
print(yhat * x_data_mask)

[[6. 7. 1. 0. 0. 0. 0. 0. 0. 0.]
 [4. 7. 2. 1. 0. 0. 0. 0. 0. 0.]
 [4. 7. 3. 4. 5. 1. 4. 0. 0. 0.]
 [4. 7. 2. 1. 7. 0. 0. 0. 0. 0.]]


In [12]:
np.squeeze(y_data, axis=-1)

array([[6, 7, 1, 0, 0, 0, 0, 0, 0, 0],
       [4, 7, 2, 1, 0, 0, 0, 0, 0, 0],
       [4, 7, 3, 4, 5, 1, 4, 0, 0, 0],
       [4, 7, 2, 1, 7, 0, 0, 0, 0, 0]], dtype=int32)